## Import library

In [ ]:
from bayes_opt import BayesianOptimization, UtilityFunction
from scipy.stats import qmc

## Objective function

In [2]:
# Design a reactor with the provide parameters
# Measure the RTD
def rtd():
    while True:
        try:
            rtd_value = float(input('Enter RTD value: '))
            return -rtd_value # Nagative to minimize RTD
        except ValueError:
            print("Invalid input. Please enter a numeric RTD value.")

## Define parameter bounds

In [3]:
# inlet_angle degree 0-60
#  aspect_ratio width/length
# RT inlet position
pbounds = {'inlet_angle':(0,60),
           'aspect':(0.2,0.6),
           'residence_time':(1,10),
          }

## Define optimizer

In [4]:
optimizer = BayesianOptimization(
    f=None, # Function is manually evaluated through user input
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

In [5]:
# 7 initial points 
initial_data = [
    {'inlet_angle': 30, 'aspect': 0.24, 'residence_time': 2, 'variance': -0.103},
    {'inlet_angle': 60, 'aspect': 0.24, 'residence_time': 6, 'variance': -0.158},
    {'inlet_angle': 50, 'aspect': 0.40, 'residence_time': 5, 'variance': -0.157},
    {'inlet_angle': 0, 'aspect': 0.40, 'residence_time': 1, 'variance': -0.367},
    {'inlet_angle': 0, 'aspect': 0.30, 'residence_time': 3, 'variance': -0.220},
    {'inlet_angle': 36, 'aspect': 0.35, 'residence_time': 8, 'variance': -0.433},
    {'inlet_angle': 24, 'aspect': 0.30, 'residence_time': 10, 'variance': -0.207}
]


## Initial loop

In [6]:
# Register predefined initial data points
for data in initial_data:
    params = {k: data[k] for k in ['inlet_angle', 'aspect', 'residence_time']}
    target = data['variance']
    
    print("Registering initial point:", params, "with variance:", target)
    
    optimizer.register(params=params, target=target)


Registering initial point: {'inlet_angle': 30, 'aspect': 0.24, 'residence_time': 2} with variance: -0.103
Registering initial point: {'inlet_angle': 60, 'aspect': 0.24, 'residence_time': 6} with variance: -0.158
Registering initial point: {'inlet_angle': 50, 'aspect': 0.4, 'residence_time': 5} with variance: -0.157
Registering initial point: {'inlet_angle': 0, 'aspect': 0.4, 'residence_time': 1} with variance: -0.367
Registering initial point: {'inlet_angle': 0, 'aspect': 0.3, 'residence_time': 3} with variance: -0.22
Registering initial point: {'inlet_angle': 36, 'aspect': 0.35, 'residence_time': 8} with variance: -0.433
Registering initial point: {'inlet_angle': 24, 'aspect': 0.3, 'residence_time': 10} with variance: -0.207


In [7]:
# 5 points initial iteration
# num_init = 5
# utility = UtilityFunction(kind='ei', xi=0.1)
# for _ in range(num_init):
#     next_point_to_probe = optimizer.suggest(utility)
#     print('Reaction design conditions: ', next_point_to_probe)
#     target = rtd()
#     print(f'RTD value: {target}')

#     # update model
#     optimizer.register(
#         params=next_point_to_probe,
#         target=target,
#     )

## Bayesian loop

In [8]:
# 10 points Bayesian loop
# num_iterations = 10
# for _ in range(num_iterations):
#     next_point_to_probe = optimizer.suggest(utility)
#     print('Reaction design conditions: ', next_point_to_probe)
#     target = rtd()
#     print(f'RTD value: {target}')

#     # update model
#     optimizer.register(
#         params=next_point_to_probe,
#         target=target,
#     )

In [ ]:
# Define Utility Function (Encourage faster exploration)
utility = UtilityFunction(kind='ei', xi=0.1)  # Expected Improvement (EI)

# Start Bayesian Optimization Loop
num_iterations = 10  # Increase if needed

for _ in range(num_iterations):
    next_point_to_probe = optimizer.suggest(utility)
    print("Suggested next design:", next_point_to_probe)

    # Obtain RTD value (manual input or from a function)
    target = float(input("Enter RTD value for this design: "))
    print(f"RTD value: {target}")

    # Register new experimental result
    optimizer.register(params=next_point_to_probe, target=-target)  # Minimize RTD

Suggested next design: {'aspect': 0.5401667412372526, 'inlet_angle': 14.23189572951146, 'residence_time': 9.599563842514037}


Enter RTD value for this design:  0.067


RTD value: 0.067
Suggested next design: {'aspect': 0.4833041522552059, 'inlet_angle': 15.076075594054663, 'residence_time': 9.227912452218613}


Enter RTD value for this design:  0.19


RTD value: 0.19
Suggested next design: {'aspect': 0.41618389567419867, 'inlet_angle': 13.590978989401611, 'residence_time': 9.876838340040175}


Enter RTD value for this design:  0.063


RTD value: 0.063
Suggested next design: {'aspect': 0.6, 'inlet_angle': 13.167019096158985, 'residence_time': 8.559428461011267}


## Plot 